# Iterating through all patients in all years to find peep>5

In [1]:
import pandas as pd
from datetime import datetime
import os
import pickle
from datetime import timedelta
from datetime import datetime
from datetime import date
from datetime import time
from dateutil.relativedelta import relativedelta



In [2]:
years = ['2014', '2015', '2016', '2017', '2018', '2019', '2020']
peepMerged = pd.DataFrame()
Sep_Vent_Pat_dur_Merged = pd.DataFrame()
Sep_Pat_file = pd.read_csv('/labs/kamaleswaranlab/MODS/cemars4/ARDS/Sep_Loc_Pat_ID_1_5_22.csv')
Sep_Pat_file['pat_id'] = Sep_Pat_file['PAT_ID']
count=0
for year in years:
    ventFilePath = '/labs/kamaleswaranlab/MODS/Data/Emory_Data/em_data/'+str(year)+'/CJSEPSIS_VENT_'+str(year)+'.dsv'
    if not os.path.isfile(ventFilePath):
        print(f"Not in the path: {ventFilePath}")
        break
    else:
#         print(f"found {ventFilePath}")
        
        VentPat = pd.read_csv(ventFilePath, sep='|').dropna().reset_index()
        #count+=len(list(pd.unique(peepFiltered['csn'])))
        
        Sep_Vent_Pat = Sep_Pat_file.merge(VentPat,on='pat_id')
        Sep_Vent_Pat_af = Sep_Vent_Pat[Sep_Vent_Pat['BED_LOCATION_START']<=Sep_Vent_Pat['recorded_time']]
        Sep_Vent_Pat_dur = Sep_Vent_Pat_af[Sep_Vent_Pat_af['BED_LOCATION_END']>=Sep_Vent_Pat_af['recorded_time']]
        Sep_Vent_Pat_dur['year'] = year
        Sep_Vent_Pat_dur_Merged = pd.concat([Sep_Vent_Pat_dur_Merged, Sep_Vent_Pat_dur], axis=0)
        #peepFiltered['peep'] = pd.to_numeric(peepFiltered['peep'], errors='coerce')
        #peepFiltered = peepFiltered[pd.to_numeric(peepFiltered['peep'])>=5][['pat_id', 'csn','peep', 'recorded_time', 'fio2']]
        #peepFiltered['year'] = year
        #peepMerged = pd.concat([peepMerged, peepFiltered], axis=0)
        
#print(f'No of total encounters in the vent files={count}')        


/opt/rh/rh-python36/root/usr/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (9) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/opt/rh/rh-python36/root/usr/lib/python3.6/site-packages/ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/opt/rh/rh-python36/root/usr/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (7,8,10) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/opt/rh/rh-python36/root/usr/lib/python3.6/site-packages/ipykernel_launcher.py:21: SettingWithCopyWarnin

# Extracting data for all patients with peep>5 and p/f<300 or s/f<315:

In [4]:
csns = list(pd.unique(Sep_Vent_Pat_dur_Merged['csn']))
print(f'Number of patients/beds with Sepsis = {len(csns)}')
count=0
countARDS=0
merged2 = pd.DataFrame()
ARDS_TS = pd.DataFrame(columns=['csn','ARDS_TS'])
for csn in csns:
    year = pd.unique(Sep_Vent_Pat_dur_Merged[Sep_Vent_Pat_dur_Merged['csn']==csn]['year'].values[0])[0]
    pkl_path = '/labs/kamaleswaranlab/MODS/Encounter_Pickles/em/'+year+'/'+str(csn)+'.pickle'
    # print(pkl_path)
    if not os.path.isfile(pkl_path):
#         print(f"Not in the path: {pkl_path}")
        count+=1
#         break
    else:
        # print(f"found {pkl_path}")
    
        with open(pkl_path, 'rb') as pf:
            patient_enc_dict = pickle.load(pf)
        sys.exit()
        super_table = patient_enc_dict['super_table']
        if len(super_table[super_table['pf_pa']<=300])>0:
            countARDS+=1
            ARDS_ts = super_table[super_table['pf_pa']<=300].index[0].strftime("%m/%d/%Y, %H:%M:%S")
            ARDS_TS=ARDS_TS.append({'csn': csn,'ARDS_TS':ARDS_ts}, ignore_index=True)
        #super_table1 = super_table[super_table['pf_pa']<300]
        #super_table1.index[0]
        # print(patient_enc_dict.keys())
        # print(patient_enc_dict['abx_order_time'])
        # # # print(patient_enc_dict['static_features']['age'])
        # # # print(patient_enc_dict['labs_staging'])
        # # print(patient_enc_dict['vitals_staging'][['temperature','map_line','map_cuff','pulse','unassisted_resp_rate', 'spo2', 'end_tidal_co2', 'o2_flow_rate']])
        # import sys
        # sys.exit()
        
        #icd10=patient_enc_dict['diagnosis_PerCSN']['dx_code_icd10'].values
        #icd9=patient_enc_dict['diagnosis_PerCSN']['dx_code_icd9'].values

        #if any('518' in code for code in icd9) or 'J80' in icd10:
            #super_table = patient_enc_dict['super_table']
            #super_table1 = super_table[super_table['pf_pa']<300]
            #super_table2=super_table[super_table['pf_sp']<=315]
            #super_table = pd.concat([super_table1, super_table2], axis=0)
            #super_table = super_table.drop_duplicates()
            #super_table['csn'] = csn
            
            # super_table = pd.concat([super_table, patient_enc_dict['vent_status']], axis=1)
            # print(super_table)
            # import sys
            # sys.exit()
            # labs = patient_enc_dict['labs_staging'][['anion_gap','lactate_dehydrogenase','bicarb_(hco3)','blood_urea_nitrogen_(bun)','glucose','hematocrit','albumin','bilirubin_direct','fio2', 'partial_pressure_of_carbon_dioxide_(paco2)','partial_pressure_of_oxygen_(pao2)', 'ph','saturation_of_oxygen_(sao2)','bilirubin_total', 'calcium', 'calcium_adjusted','calcium_ionized', 'chloride','phosphorus', 'potassium', 'sodium','hemoglobin','white_blood_cell_count', 'carboxy_hgb','creatinine']]
            # vitals = patient_enc_dict['vitals_staging'][['temperature','map_line','map_cuff','sbp_line','sbp_cuff','pulse','unassisted_resp_rate', 'spo2', 'end_tidal_co2', 'o2_flow_rate']]
            
            # all_measures = pd.concat([labs, vitals], axis=1)
            #merged2 = pd.concat([merged2, super_table], axis = 0)
        #else:
            #continue
        
#print(merged2)

#merged2.to_csv('eARDSData.csv')

#print(f'Number of patients with s/f<=315 and p/f<300 = {len(list(pd.unique(merged2["csn"])))}')
#print(f'Number of encounters not found in the encounter pickles: {count}')



Number of patients/beds with Sepsis = 2804


NameError: name 'sys' is not defined

In [3]:
countARDS

NameError: name 'countARDS' is not defined

In [6]:
ARDS_ts

'07/29/2019, 00:12:04'

In [5]:
ARDS_TS

,csn,ARDS_TS
0,3507474226,"08/27/2014, 10:47:00"
1,4038504074,"03/15/2014, 20:13:10"
2,45660724103,"06/07/2014, 21:52:00"
3,7683234230,"09/16/2014, 16:19:00"
4,4387424019,"01/20/2014, 00:21:05"
...,...,...
2088,57735529284,"10/11/2019, 20:52:00"
2089,57751399297,"10/24/2019, 13:09:04"
2090,57845139057,"03/06/2019, 04:33:54"
2091,57982189346,"12/23/2019, 10:59:00"


In [7]:
Sep_Pat_file.columns

Index(['PAT_ID', 'BED_LOCATION_START', 'BED_LOCATION_END', 'TSEPSIS',
       'deBedTime', 'pat_id'],
      dtype='object')

In [8]:
Sep_Vent_Pat_dur.columns

Index(['PAT_ID', 'BED_LOCATION_START', 'BED_LOCATION_END', 'TSEPSIS',
       'deBedTime', 'pat_id', 'index', 'csn', 'recorded_time',
       'vent_start_time', 'vent_stop_time', 'vent_type', 'vent_mode',
       'vent_rate_set', 'vent_tidal_rate_set', 'vent_tidal_rate_exhaled',
       'peep', 'fio2', 'year'],
      dtype='object')

In [9]:
Sep_Vent_Pat_dur.head()

,PAT_ID,BED_LOCATION_START,BED_LOCATION_END,TSEPSIS,deBedTime,pat_id,index,csn,recorded_time,vent_start_time,vent_stop_time,vent_type,vent_mode,vent_rate_set,vent_tidal_rate_set,vent_tidal_rate_exhaled,peep,fio2,year
469,100242766,03/03/2020 20:46:08,03/08/2020 15:09:01,2010-03-30 16:45:00,A140-0583286241,100242766,152166,58483030054,03/03/2020 21:50:00,03/03/2020 21:50:00,03/08/2020 20:37:00,Hamilton C3,AC/CMV Volume,24,410,438,6,1.0,2020
470,100242766,03/03/2020 20:46:08,03/08/2020 15:09:01,2010-03-30 16:45:00,A140-0583286241,100242766,152169,58483030054,03/04/2020 00:57:00,03/03/2020 21:50:00,03/08/2020 20:37:00,Hamilton C3,AC/CMV Volume,35,410,400,12,0.80,2020
471,100242766,03/03/2020 20:46:08,03/08/2020 15:09:01,2010-03-30 16:45:00,A140-0583286241,100242766,152172,58483030054,03/04/2020 04:11:00,03/03/2020 21:50:00,03/08/2020 20:37:00,Hamilton C3,AC/CMV Volume,35,410,400,12,0.75,2020
472,100242766,03/03/2020 20:46:08,03/08/2020 15:09:01,2010-03-30 16:45:00,A140-0583286241,100242766,152174,58483030054,03/04/2020 08:08:00,03/03/2020 21:50:00,03/08/2020 20:37:00,Hamilton C3,AC/CMV Volume,35,450,434,12,0.7,2020
473,100242766,03/03/2020 20:46:08,03/08/2020 15:09:01,2010-03-30 16:45:00,A140-0583286241,100242766,152175,58483030054,03/04/2020 11:38:00,03/03/2020 21:50:00,03/08/2020 20:37:00,Hamilton C3,AC/CMV Volume,35,450,436,12,0.7,2020


In [10]:
Sep_ARDS_deBedTime = ARDS_TS.merge(Sep_Vent_Pat_dur_Merged,on='csn')
Sep_ARDS_deBedTime1 = Sep_ARDS_deBedTime[Sep_ARDS_deBedTime['BED_LOCATION_START']<=Sep_ARDS_deBedTime['ARDS_TS']]
Sep_ARDS_deBedTime2 = Sep_ARDS_deBedTime1[Sep_ARDS_deBedTime1['BED_LOCATION_END']>=Sep_ARDS_deBedTime1['ARDS_TS']]
Sep_ARDS_deBedTime3 = Sep_ARDS_deBedTime2[['deBedTime','BED_LOCATION_START','BED_LOCATION_END','TSEPSIS','ARDS_TS','vent_start_time']]
Sep_ARDS_deBedTime4 = Sep_ARDS_deBedTime3.drop_duplicates()
Sep_ARDS_deBedTime4

,deBedTime,BED_LOCATION_START,BED_LOCATION_END,TSEPSIS,ARDS_TS,vent_start_time
10,A050-0394924561,03/15/2014 19:00:00,03/19/2014 15:00:21,2004-04-11 06:16:00,"03/15/2014, 20:13:10",03/15/2014 21:10:00
18,A091-0409349409,09/16/2014 16:22:25,09/17/2014 04:21:00,2004-10-12 20:16:00,"09/16/2014, 16:19:00",08/29/2014 18:15:00
19,A014-0410899065,09/16/2014 16:22:25,09/17/2014 04:21:00,2004-10-12 20:16:00,"09/16/2014, 16:19:00",08/29/2014 18:15:00
21,A050-0390197026,01/20/2014 00:49:18,01/23/2014 18:21:48,2004-02-18 06:26:00,"01/20/2014, 00:21:05",01/19/2014 23:38:00
46,A014-0407529610,08/08/2014 16:27:24,08/11/2014 15:36:00,2004-09-03 16:30:00,"08/08/2014, 16:48:33",08/08/2014 16:30:00
...,...,...,...,...,...,...
90533,A090-0551197615,03/06/2019 00:57:48,03/18/2019 16:45:00,2009-04-02 15:01:00,"03/06/2019, 04:33:54",03/07/2019 17:00:00
90576,A062-0551851966,03/06/2019 00:57:48,03/18/2019 16:45:00,2009-04-02 15:01:00,"03/06/2019, 04:33:54",03/07/2019 17:00:00
90622,A138-0577157125,12/23/2019 23:47:26,12/30/2019 16:55:31,2010-01-18 14:13:00,"12/23/2019, 10:59:00",12/23/2019 12:25:00
90650,A006-0564362063,07/28/2019 21:03:41,08/31/2019 01:44:59,2009-08-24 01:32:00,"07/29/2019, 00:12:04",07/28/2019 20:40:00


In [12]:
print(len(Sep_ARDS_deBedTime4[(pd.to_datetime(Sep_ARDS_deBedTime4['ARDS_TS']) - timedelta(hours=6))>=pd.to_datetime(Sep_ARDS_deBedTime4['vent_start_time'])]))

485


In [34]:
pd.to_datetime(Sep_ARDS_deBedTime4['ARDS_TS']) + timedelta(hours=6)

10      2014-03-16 02:13:10
18      2014-09-16 22:19:00
19      2014-09-16 22:19:00
21      2014-01-20 06:21:05
46      2014-08-08 22:48:33
                ...        
90533   2019-03-06 10:33:54
90576   2019-03-06 10:33:54
90622   2019-12-23 16:59:00
90650   2019-07-29 06:12:04
90653   2019-07-29 06:12:04
Name: ARDS_TS, Length: 2703, dtype: datetime64[ns]

In [9]:
Sep_Vent_Pat_dur_Merged.columns

Index(['PAT_ID', 'BED_LOCATION_START', 'BED_LOCATION_END', 'TSEPSIS',
       'deBedTime', 'pat_id', 'index', 'csn', 'recorded_time',
       'vent_start_time', 'vent_stop_time', 'vent_type', 'vent_mode',
       'vent_rate_set', 'vent_tidal_rate_set', 'vent_tidal_rate_exhaled',
       'peep', 'fio2'],
      dtype='object')

In [14]:
patient_enc_dict.columns

AttributeError: 'dict' object has no attribute 'columns'

In [8]:
print(len(super_table))
super_table['pf_pa']

1941


2019-07-28 20:12:04      NaN
2019-07-28 21:12:04      NaN
2019-07-28 22:12:04      NaN
2019-07-28 23:12:04      NaN
2019-07-29 00:12:04    256.0
                       ...  
2019-10-17 12:12:04      NaN
2019-10-17 13:12:04      NaN
2019-10-17 14:12:04      NaN
2019-10-17 15:12:04      NaN
2019-10-17 16:12:04      NaN
Freq: H, Name: pf_pa, Length: 1941, dtype: float64

In [10]:
super_table1['pf_pa'][0:5]

2019-07-29 00:12:04    256.0
2019-07-29 01:12:04    256.0
2019-07-29 02:12:04    256.0
2019-07-29 03:12:04    256.0
2019-07-29 04:12:04    256.0
Name: pf_pa, dtype: float64

In [14]:
super_table1.index[0]

Timestamp('2019-07-29 00:12:04')

In [18]:
super_table[super_table['pf_pa']<=300].index[0].strftime("%m/%d/%Y, %H:%M:%S")

'07/29/2019, 00:12:04'

In [21]:
len(super_table[super_table['pf_pa']<=300])

0